In [11]:
from pyspark.mllib.feature import HashingTF, IDF
from pyspark.mllib.regression import LabeledPoint
from pyspark.mllib.classification import NaiveBayes   
import pyspark

try: 
    type(sc)
except NameError:
    sc = pyspark.SparkContext('local[*]') 
    
training_raw = sc.textFile('data/train.csv')


# Split data into labels and features, transform
# preservesPartitioning is not really required
# since map without partitioner shouldn't trigger 
print("Extraxendo predicciones...")
labels = training_raw.map(
    lambda doc: doc["Prediction"],  # Standard Python dict access 
    preservesPartitioning=True # This is obsolete.
)
print("OK\n")

print("Hasheando palabras...")
tf = HashingTF(numFeatures=100).transform(training_raw.map(lambda doc: doc["Text"].split(), preservesPartitioning=True))
print("OK\n")

idf = IDF().fit(tf)
tfidf = idf.transform(tf)

# Combine using zip
print("Combining using zip(?)...")
training = labels.zip(tfidf).map(lambda x: LabeledPoint(x[0], x[1]))
print("OK\n")

# Train and check
print("Trainning...")
model = NaiveBayes.train(training)
print("OK\n")

labels_and_preds = labels.zip(model.predict(tfidf)).map(lambda x: {"actual": x[0], "predicted": float(x[1])})

Extraxendo predicciones...
OK

Hasheando palabras...
OK



Py4JJavaError: An error occurred while calling o342.fitIDF.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 2 in stage 32.0 failed 1 times, most recent failure: Lost task 2.0 in stage 32.0 (TID 99, localhost): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/usr/local/spark/python/lib/pyspark.zip/pyspark/worker.py", line 111, in main
    process()
  File "/usr/local/spark/python/lib/pyspark.zip/pyspark/worker.py", line 106, in process
    serializer.dump_stream(func(split_index, iterator), outfile)
  File "/usr/local/spark/python/lib/pyspark.zip/pyspark/serializers.py", line 263, in dump_stream
    vs = list(itertools.islice(iterator, batch))
  File "<ipython-input-11-6b74a02c5fa9>", line 25, in <lambda>
TypeError: string indices must be integers

	at org.apache.spark.api.python.PythonRunner$$anon$1.read(PythonRDD.scala:166)
	at org.apache.spark.api.python.PythonRunner$$anon$1.<init>(PythonRDD.scala:207)
	at org.apache.s